In [32]:
import numpy as np 
import pandas as pd
import re
import pickle
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files 

from sklearn.preprocessing import LabelEncoder
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

In [2]:
df=pd.read_csv('E:/Chrome/134715_320111_compressed_IMDB Dataset/IMDB Dataset.csv')


In [3]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [4]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [5]:
(df.review.str.len()==0).any()

False

## using label encoder converting sentiments to  0's and 1's

In [6]:
label_enc=LabelEncoder()
df.sentiment=label_enc.fit_transform(df.sentiment)

In [7]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0


## removing HTML tags

In [8]:
def remove_html(text):
    soup=BeautifulSoup(text,"html.parser")
    return soup.get_text()

In [9]:
df.review=df.review.apply(remove_html)

In [10]:
df.review

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. The filming tec...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

## remove everything except lower/upper case letters 

In [11]:
def rem_extra(text):
    text=re.sub(r'\[[^]]*\]', '', text)
    text=re.sub(r'[^a-zA-Z]', ' ', text)
    text=re.sub(r'\s+[a-z]\s+',' ',text)
    text=re.sub(r'[^a-zA-Z]\s+',' ',text)
    text=re.sub(r'\s+',' ',text)
    return text



In [12]:
df.review=df.review.apply(rem_extra)

In [13]:
df.review

0        One of the other reviewers has mentioned that ...
1        A wonderful little production The filming tech...
2        I thought this was wonderful way to spend time...
3        Basically there a family where little boy Jake...
4        Petter Mattei Love in the Time of Money is vis...
                               ...                        
49995    I thought this movie did down right good job I...
49996    Bad plot bad dialogue bad acting idiotic direc...
49997    I am Catholic taught in parochial elementary s...
49998    I going to have to disagree with the previous ...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

## Converting to lower case

In [14]:
def conv_lower(text):
    text=text.lower()
    return text
df.review=df.review.apply(conv_lower)

In [15]:
df.review

0        one of the other reviewers has mentioned that ...
1        a wonderful little production the filming tech...
2        i thought this was wonderful way to spend time...
3        basically there a family where little boy jake...
4        petter mattei love in the time of money is vis...
                               ...                        
49995    i thought this movie did down right good job i...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    i am catholic taught in parochial elementary s...
49998    i going to have to disagree with the previous ...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

## Removing Stop words

In [16]:
stop_words=set(stopwords.words('english'))
def rm_stopwords(text):
    text=[word for word in text.split() if not word in stop_words]
    return text
    

In [17]:
df.review=df.review.apply(rm_stopwords)

In [18]:
df.review

0        [one, reviewers, mentioned, watching, oz, epis...
1        [wonderful, little, production, filming, techn...
2        [thought, wonderful, way, spend, time, hot, su...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [thought, movie, right, good, job, creative, o...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [catholic, taught, parochial, elementary, scho...
49998    [going, disagree, previous, comment, side, mal...
49999    [one, expects, star, trek, movies, high, art, ...
Name: review, Length: 50000, dtype: object

In [19]:
def lemma_review(text):
    lem = WordNetLemmatizer()
    text= ' '.join([lem.lemmatize(word) for word in text])
    return text

In [20]:
df.review=df.review.apply(lemma_review)

In [21]:
df.review

0        one reviewer mentioned watching oz episode hoo...
1        wonderful little production filming technique ...
2        thought wonderful way spend time hot summer we...
3        basically family little boy jake think zombie ...
4        petter mattei love time money visually stunnin...
                               ...                        
49995    thought movie right good job creative original...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    catholic taught parochial elementary school nu...
49998    going disagree previous comment side maltin on...
49999    one expects star trek movie high art fan expec...
Name: review, Length: 50000, dtype: object

In [22]:
X=df.review
y=df.sentiment

In [25]:
from sklearn.pipeline import Pipeline

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
text_clf_nb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB()),
])

# Linear SVC:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

In [28]:
text_clf_nb.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [29]:
predictions = text_clf_nb.predict(X_test)

In [30]:
print(confusion_matrix(y_test,predictions))

[[7196 1012]
 [1245 7047]]


In [33]:
print(accuracy_score(y_test,predictions))

0.8632121212121212


In [34]:
text_clf_lsvc.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [35]:
predictions = text_clf_lsvc.predict(X_test)

In [36]:
print(confusion_matrix(y_test,predictions))

[[7256  952]
 [ 768 7524]]


In [37]:
print(accuracy_score(y_test,predictions))

0.8957575757575758
